# data augmentation 
- use FFmpeg （https://github.com/FFmpeg/FFmpeg）for filp and color jitter
- generate new data for all mp4

In [2]:
import subprocess
import os
import random

def generate_random_color_jitter():
    """
    随机生成色彩调整参数，包括亮度、对比度、饱和度和伽玛。
    :return: 一个包含色彩调整参数的字符串，可以直接用于 FFmpeg 的 eq 滤镜。
    """
    # 随机生成 brightness, contrast, saturation, gamma 的值
    brightness = round(random.uniform(-0.1, 0.1), 2)  # 亮度范围：-0.1 到 0.1
    contrast = round(random.uniform(0.8, 1.5), 2)     # 对比度范围：0.8 到 1.5
    saturation = round(random.uniform(0.8, 1.5), 2)   # 饱和度范围：0.8 到 1.5
    gamma = round(random.uniform(0.8, 1.2), 2)        # 伽玛范围：0.8 到 1.2
    hue = round(random.uniform(-30, 30), 2)           # 色调范围：-30 到 30 (单位：度)
    
    # 构造 eq 和 hue 滤镜参数字符串
    eq_params = f"eq=brightness={brightness}:contrast={contrast}:saturation={saturation}:gamma={gamma}"
    hue_params = f"hue=h={hue}:s=1"
    
    return f"{eq_params},{hue_params}"

def flip_and_color_adjust_all(input_dir, output_dir, log_file):
    """
    批量处理 input_dir 目录中的所有 mp4 文件，应用水平翻转和随机色彩调整，并将结果保存到 output_dir 中。
    输出文件按 <原文件名>_flip_colorjitter.mp4 命名，处理日志输出到 log_file。
    
    :param input_dir: 输入视频文件夹路径
    :param output_dir: 输出文件夹路径
    :param log_file: 用于保存输出日志的文件路径
    """
    
    # 检查输入文件夹是否存在
    if not os.path.exists(input_dir):
        with open(log_file, 'a', encoding='utf-8') as log:
            log.write(f"输入文件夹 {input_dir} 不存在\n")
        return
    
    # 如果输出文件夹不存在，创建它
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
        with open(log_file, 'a', encoding='utf-8') as log:
            log.write(f"输出文件夹 {output_dir} 已创建\n")

    # 打开日志文件进行写入，使用 UTF-8 编码
    with open(log_file, 'a', encoding='utf-8') as log:
        # 遍历输入文件夹中的所有 mp4 文件
        for filename in os.listdir(input_dir):
            if filename.endswith(".mp4"):
                input_file = os.path.join(input_dir, filename)
                
                # 构造输出文件名：<原文件名>_flip_colorjitter.mp4
                base_name = os.path.splitext(filename)[0]
                output_filename = f"{base_name}_flip_colorjitter.mp4"
                output_file = os.path.join(output_dir, output_filename)
                
                # 生成随机的色彩调整参数
                color_jitter_params = generate_random_color_jitter()

                # 构建 FFmpeg 命令，水平翻转并应用随机色彩调整
                command = [
                    "ffmpeg",           # FFmpeg 可执行文件
                    "-i", input_file,   # 输入文件
                    "-vf", f"hflip,{color_jitter_params}",  # 水平翻转和随机色彩调整滤镜
                    "-c:v", "libx264",  # 使用 H.264 编码器重新编码视频
                    "-crf", "23",       # CRF 质量参数 (可根据需要调整)
                    "-preset", "medium",# 编码速度/压缩比
                    "-c:a", "copy",     # 复制音频流而不重新编码
                    output_file         # 输出文件
                ]

                try:
                    # 执行 FFmpeg 命令
                    result = subprocess.run(command, check=True, stderr=subprocess.PIPE, stdout=subprocess.PIPE, text=True)

                    # 将处理成功的信息写入日志文件
                    log.write(f"视频已成功处理: {input_file}\n")
                    log.write(f"保存为: {output_file}\n")
                    log.write(f"使用的色彩参数: {color_jitter_params}\n\n")
                    
                except subprocess.CalledProcessError as e:
                    # 如果处理出错，将错误信息写入日志文件
                    log.write(f"处理文件 {input_file} 时发生错误: {e.stderr}\n")
                except FileNotFoundError:
                    log.write("FFmpeg 未找到，请确保 FFmpeg 安装并添加到系统 PATH 中。\n")

# 示例用法
input_directory = r"D:\Test"  # 输入文件夹路径
output_directory = r"D:\Test_1"  # 输出文件夹路径
log_file_path = r"D:\adjustment.txt"  # 日志文件路径

# 调用函数进行批量处理
flip_and_color_adjust_all(input_directory, output_directory, log_file_path)

In [4]:
# 示例用法
input_directory = r"D:\Train"  # 输入文件夹路径
output_directory = r"D:\Train_1"  # 输出文件夹路径
log_file_path = r"D:\adjustment_train.txt"  # 日志文件路径

# 调用函数进行批量处理
flip_and_color_adjust_all(input_directory, output_directory, log_file_path)

In [6]:
# 示例用法
input_directory = r"D:\Validation"  # 输入文件夹路径
output_directory = r"D:\Validation_1"  # 输出文件夹路径
log_file_path = r"D:\adjustment_validation.txt"  # 日志文件路径

# 调用函数进行批量处理
flip_and_color_adjust_all(input_directory, output_directory, log_file_path)